In [1]:
!pip install git+https://github.com/guillaumegenthial/tf_metrics.git

  Cloning https://github.com/guillaumegenthial/tf_metrics.git to /tmp/pip-req-build-q302ah2h
  Running command git clone -q https://github.com/guillaumegenthial/tf_metrics.git /tmp/pip-req-build-q302ah2h
  Created wheel for tf-metrics: filename=tf_metrics-0.0.1-cp36-none-any.whl size=7694 sha256=7d6e63b15c36b03edf22812ed4b2e44a47baa379688519186a7da61f5a9cbfc0
  Stored in directory: /tmp/pip-ephem-wheel-cache-hry__ffb/wheels/da/6c/c8/663ef339a0666590dc53bd13bab86643a1f9c35b26742d7876
Successfully built tf-metrics


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/mhc/')

In [4]:
%cd "/content/drive/My Drive/Colab Notebooks/mhc/"

/content/drive/My Drive/Colab Notebooks/mhc


In [5]:
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

TensorFlow 1.x selected.
1.15.2


In [6]:
import functools
import logging
from tf_metrics import precision, recall, f1

In [1]:
from pathlib import Path
import sys
import numpy as np
import json

from bilstm import BiLSTM
from buildGloveVocab import GloveVocab

In [ ]:
GloveVocab.create_words_file(data_dir+"histoclass/train.txt", data_dir+"histoclass")
GloveVocab.create_words_file(data_dir+"histoclass/test.txt", data_dir+"histoclass", "test")
GloveVocab.create_words_file(data_dir+"histoclass/dev.txt", data_dir+"histoclass", "dev")
GloveVocab.create_tags_file(data_dir+"histoclass/train.txt", data_dir+"histoclass")
GloveVocab.create_tags_file(data_dir+"histoclass/test.txt", data_dir+"histoclass", "test")
GloveVocab.create_tags_file(data_dir+"histoclass/dev.txt", data_dir+"histoclass", "dev")

In [ ]:
GloveVocab.create_words_file(data_dir+"histomention/train.txt", data_dir+"histomention")
GloveVocab.create_words_file(data_dir+"histomention/test.txt", data_dir+"histomention", "test")
GloveVocab.create_words_file(data_dir+"histomention/dev.txt", data_dir+"histomention", "dev")
GloveVocab.create_tags_file(data_dir+"histomention/train.txt", data_dir+"histomention")
GloveVocab.create_tags_file(data_dir+"histomention/test.txt", data_dir+"histomention", "test")
GloveVocab.create_tags_file(data_dir+"histomention/dev.txt", data_dir+"histomention", "dev")

In [ ]:
data_paths = [data_dir+"histoclass/train", data_dir+"histoclass/dev", data_dir+"histoclass/test"]

In [ ]:
GloveVocab.build_vocab(data_paths)

Build vocab words (may take a while)
- done. Kept 9825 out of 9825
Build vocab chars
- done. Found 100 chars
Build vocab tags (may take a while)
- done. Found 45 tags.


In [ ]:
GloveVocab.build_glove(path_vocab_words='data/histo/histoclass', path_glove_txt="wordemb/glove.840B.300d.txt")

Reading GloVe file (may take a while)
- At line 0
- At line 100000
- At line 200000
- At line 300000
- At line 400000
- At line 500000
- At line 600000
- At line 700000
- At line 800000
- At line 900000
- At line 1000000
- At line 1100000
- At line 1200000
- At line 1300000
- At line 1400000
- At line 1500000
- At line 1600000
- At line 1700000
- At line 1800000
- At line 1900000
- At line 2000000
- At line 2100000
- done. Found 9625 vectors for 9825 words


In [ ]:
DATADIR = 'data/histo/histoclass'

# Logging
Path('results').mkdir(exist_ok=True)
tf.compat.v1.logging.set_verbosity(logging.INFO)
handlers = [
    logging.FileHandler('results/main.log'),
    logging.StreamHandler(sys.stdout)
]
logging.getLogger('tensorflow').handlers = handlers

In [ ]:
    params = {
        'dim': 300,
        'dropout': 0.5,
        'num_oov_buckets': 1,
        'epochs': 25,
        'batch_size': 20,
        'buffer': 15000,
        'lstm_size': 100,
        'words': str(Path(DATADIR, 'vocab.words.txt')),
        'chars': str(Path(DATADIR, 'vocab.chars.txt')),
        'tags': str(Path(DATADIR, 'vocab.tags.txt')),
        'glove': str(Path(DATADIR, 'glove.npz'))
    }
    with Path('results/params.json').open('w') as f:
        json.dump(params, f, indent=4, sort_keys=True)

    
    # Estimator, train and evaluate
    train_inpf = functools.partial(BiLSTM.input_fn, BiLSTM.fwords('train', DATADIR), BiLSTM.ftags('train', DATADIR),
                                   params, shuffle_and_repeat=True)
    eval_inpf = functools.partial(BiLSTM.input_fn, BiLSTM.fwords('test', DATADIR), BiLSTM.ftags('test', DATADIR))

    cfg = tf.estimator.RunConfig(save_checkpoints_secs=120)
    estimator = tf.estimator.Estimator(BiLSTM.model_fn, 'results/model', cfg, params)
    Path(estimator.eval_dir()).mkdir(parents=True, exist_ok=True)
    #hook = tf.contrib.estimator.stop_if_no_increase_hook(
    #    estimator, 'f1', 500, min_steps=8000, run_every_secs=120)
    #train_spec = tf.estimator.TrainSpec(input_fn=train_inpf, hooks=[hook])
    train_spec = tf.estimator.TrainSpec(input_fn=train_inpf)
    eval_spec = tf.estimator.EvalSpec(input_fn=eval_inpf, throttle_secs=120)
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

Using config: {'_model_dir': 'results/model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 120, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4534441a20>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Not using Distribute Coordinator.
Running training and evaluation locally (non-distributed).
Start train and eval

({'acc': 0.90115476,
  'f1': 0.5642268,
  'global_step': 2480,
  'loss': 8.172856,
  'precision': 0.61587304,
  'recall': 0.5205724},
 [])

In [ ]:
for name in ['train', 'dev', 'test']:
        BiLSTM.write_predictions(name, estimator, DATADIR, "results")

Calling model_fn.
From /content/drive/My Drive/Colab Notebooks/mhc/bilstm.py:90: to_int64 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
Done calling model_fn.
Graph was finalized.
Restoring parameters from results/model/model.ckpt-2480
Running local_init_op.
Done running local_init_op.
Calling model_fn.
Done calling model_fn.
Graph was finalized.
Restoring parameters from results/model/model.ckpt-2480
Running local_init_op.
Done running local_init_op.
Calling model_fn.
Done calling model_fn.
Graph was finalized.
Restoring parameters from results/model/model.ckpt-2480
Running local_init_op.
Done running local_init_op.


In [ ]:
GloveVocab.build_glove(path_vocab_words='data/histo/histoclass',path_glove_txt="wordemb/HistoGlove.txt", output_name="glove_hist.npz")

Reading GloVe file (may take a while)
- At line 0
- At line 100000
- done. Found 7310 vectors for 9825 words


In [9]:
DATADIR = 'data/histo/histoclass'

# Logging
Path('resultshisto').mkdir(exist_ok=True)
tf.compat.v1.logging.set_verbosity(logging.INFO)
handlers = [
    logging.FileHandler('resultshisto/main.log'),
    logging.StreamHandler(sys.stdout)
]
logging.getLogger('tensorflow').handlers = handlers

In [10]:
    params = {
        'dim': 300,
        'dropout': 0.5,
        'num_oov_buckets': 1,
        'epochs': 25,
        'batch_size': 20,
        'buffer': 15000,
        'lstm_size': 100,
        'words': str(Path(DATADIR, 'vocab.words.txt')),
        'chars': str(Path(DATADIR, 'vocab.chars.txt')),
        'tags': str(Path(DATADIR, 'vocab.tags.txt')),
        'glove': str(Path(DATADIR, 'glove_hist.npz'))
    }
    with Path('resultshisto/params.json').open('w') as f:
        json.dump(params, f, indent=4, sort_keys=True)

    
    # Estimator, train and evaluate
    train_inpf = functools.partial(BiLSTM.input_fn, BiLSTM.fwords('train', DATADIR), BiLSTM.ftags('train', DATADIR),
                                   params, shuffle_and_repeat=True)
    eval_inpf = functools.partial(BiLSTM.input_fn, BiLSTM.fwords('test', DATADIR), BiLSTM.ftags('test', DATADIR))

    cfg = tf.estimator.RunConfig(save_checkpoints_secs=120)
    estimator = tf.estimator.Estimator(BiLSTM.model_fn, 'resultshisto/model', cfg, params)
    Path(estimator.eval_dir()).mkdir(parents=True, exist_ok=True)
    #hook = tf.contrib.estimator.stop_if_no_increase_hook(
    #    estimator, 'f1', 500, min_steps=8000, run_every_secs=120)
    #train_spec = tf.estimator.TrainSpec(input_fn=train_inpf, hooks=[hook])
    train_spec = tf.estimator.TrainSpec(input_fn=train_inpf)
    eval_spec = tf.estimator.EvalSpec(input_fn=eval_inpf, throttle_secs=120)
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

Using config: {'_model_dir': 'resultshisto/model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 120, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0b94edc080>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Not using Distribute Coordinator.
Running training and evaluation locally (non-distributed).
Start train and

({'acc': 0.8922248,
  'f1': 0.52380955,
  'global_step': 2480,
  'loss': 9.343881,
  'precision': 0.60397196,
  'recall': 0.46243292},
 [])

In [11]:
for name in ['train', 'dev', 'test']:
        BiLSTM.write_predictions(name, estimator, DATADIR, "resultshisto")

Calling model_fn.
From /content/drive/My Drive/Colab Notebooks/mhc/bilstm.py:90: to_int64 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
Done calling model_fn.
Graph was finalized.
Restoring parameters from resultshisto/model/model.ckpt-2480
Running local_init_op.
Done running local_init_op.
Calling model_fn.
Done calling model_fn.
Graph was finalized.
Restoring parameters from resultshisto/model/model.ckpt-2480
Running local_init_op.
Done running local_init_op.
Calling model_fn.
Done calling model_fn.
Graph was finalized.
Restoring parameters from resultshisto/model/model.ckpt-2480
Running local_init_op.
Done running local_init_op.


In [25]:
from sklearn.metrics import f1_score, precision_score, recall_score

In [35]:
def red_pred(path):
    y, y_pred = [], []
    with Path(path).open("r") as f:
        for l in f:
            if l.strip():
                words = l.split()
                y.append(words[1])
                y_pred.append(words[2])
        return y, y_pred

In [36]:
y, y_pred = red_pred("results/score/test.preds.txt")

In [37]:
f1 = f1_score(y, y_pred, average='weighted')
prec = precision_score(y, y_pred, average='weighted')
rec = recall_score(y, y_pred, average='weighted')
print(f1, prec, rec)

0.8917272518912011 0.8891516370522373 0.9011547344110854


In [38]:
y, y_pred = red_pred("resultshisto/score/test.preds.txt")

In [39]:
f1 = f1_score(y, y_pred, average='weighted')
prec = precision_score(y, y_pred, average='weighted')
rec = recall_score(y, y_pred, average='weighted')
print(f1, prec, rec)

0.878688859838361 0.8755559611760211 0.8922247882986913
